# Creation of the GALFIT mask in *.fits* format   (with the WCS coordinates of the data image)
### i.e. pixels to be ignored during the fit

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from astropy.io import fits
from astropy.nddata.utils import Cutout2D
from astropy.coordinates import SkyCoord

Steps 1-5 are taken from the tutorial on GALFIT's website: https://users.obs.carnegiescience.edu/peng/work/galfit/MASKING.html  .  
In particular the scripts `ds9poly` and `fillpoly` used in the following steps, and included in this folder, are taken from this guide.

1) Using ds9 (https://sites.google.com/cfa.harvard.edu/saoimageds9), choose a polygonal region, then save it. The format must be *ds9*, coordinates *image*. Save it with a name as *ds9out*, without the extension. It will be useful later in the situation in which one wants to mask more regions not connected. These regions has to be taken one at a time and save them with, for example, *ds9out1*,*ds9out2*, ...   
Don't worry if the region goes outside the image: the outer points will be cut in the following steps (step 7).  
        
2) Execute `./ds9poly ds9out > vertices`. If there are more regions, you will have *vertices1*, *vertices2*, ... 

3) Execute `./fillpoly vertices mask.txt`. If there are more regions, you will have *mask1.txt*, *mask2.txt*, ... 

4) If there are more regions, and there are *mask1.txt*, *mask2.txt*, ..., merge all them together in a single text file with `cat *.txt > mask` .

5) The file `mask` contains the coordinates $(x,y)$ of the pixels to be ignored during the fit. 

The next two mirrors allow to see the region, by plotting the coordinates from the text file.

------

6) In order to convert the text file into a *.fits* file, you can usa the last mirror. The bad pixels of the mask have a value $>0$ (I put them $=1$) and the good pixels have value $=0$. 
The mask will have the WCS coordinates of the data image (very useful to match frames in ds9, for example...)

7) In the second part of the last mirror, the mask has been cropped, to have the same dimension of the data image.

In [ ]:
#read the `mask` file and save the coordinates of the bad pixels
x, y = np.loadtxt('mask', dtype='int', unpack=True)

#open the image of the data. It is important if you want the mask to have the right WCS coordinates
hdu = fits.open('image_data.fits')[0]
header = hdu.header
wcs = WCS(hdu.header)

In [ ]:
#if you want to plot the mask superimposed to the data image to check it
plt.figure(figsize=(13,6))
plt.subplot(121, projection=wcs)
plt.imshow(hdu.data, cmap='gray')
plt.xlabel('Right Ascension')
plt.ylabel('declination')

plt.subplot(122, projection=wcs)
plt.imshow(hdu.data, cmap='gray')
plt.scatter(x,y)
plt.xlim(0,hdu.data.shape[0])
plt.ylim(0,hdu.data.shape[1])
plt.xlabel('Right Ascension')
plt.ylabel(' ')

plt.savefig('data&mask.jpg') #save the plot
plt.show()

In [ ]:
# the new fits image
new = np.zeros((max(y),max(x))) 

# put the bad pixels with a value >0
for i in range(len(x)):
    new[y[i]-1][x[i]-1] = 1

#create the fits file with the header of the data image
mask = fits.PrimaryHDU(new, header) 

#cropping it the same dimension of the image
# Make the cutout, including the WCS
position = SkyCoord(177.3937308, 22.40132616, unit="deg", frame="fk5") #center of the crop. Here I crop with the SkyCoord RA and dec
xbox, ybox = hdu.data.shape
cutout = Cutout2D(mask.data, position , (xbox, ybox), wcs=wcs)

# Put the cutout image in the FITS HDU
mask.data = cutout.data

# Update the FITS header with the cutout WCS
mask.header.update(cutout.wcs.to_header())

# Write the cutout to a new FITS file
mask.writeto('mask.fits', overwrite=True)